In [17]:
%load_ext autoreload
%autoreload
import os, sys

modules = os.path.abspath(os.path.join("..", "src"))
sys.path.append(modules)

import nlp_utils, keywords_utils, load_utils
from config import *
import polars as pl
from transformers import pipeline
from sentence_transformers import SentenceTransformer

from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import torch

# Load the model
model = SentenceTransformer("google/embeddinggemma-300m")

# classification_prompt = "task: classification | query: "

# lf = load_utils.load_parquets("/home/msalvetti/KOFScraper/chunks/run_id=2025-11-21/parsed").select("ID", "url", "text")
# frame = lf.slice(0, 100).collect(engine = "streaming")
# frame = frame.filter(pl.col("text")!= "")\
# .filter(~pl.col("text").str.starts_with("Powered by Imunify360")).unique("text")\
# .filter(~pl.col("text").str.contains("Loader Please wait while your request is being verified..."))\
# .filter(~pl.col("text").str.contains("You need to enable JavaScript to run this app"))\
# .filter(~pl.col("text").str.contains("flex-basis: "))\
# .filter(~pl.col("text").str.contains("document.getElementById"))
# texts = frame["text"].to_list()
# urls = frame["url"].to_list()

# labels = [
# "Advanced optical communication,Light-based data transmission via fibers or free space (e.g., lasers, Li-Fi).",
# "Advanced radiofrequency communication,Radio wave transmission with advanced antennas and beamforming (e.g., 5G/6G, satellites).",
# "Advanced undersea wireless communication,Acoustic or optical data transfer underwater.",
# "Cloud and edge computing,Distributed processing in central clouds and local devices for low-latency apps.",
# "Digital twins,Live virtual models of physical systems updated in real time.",
# "Distributed ledgers,Decentralised transaction records (e.g., blockchain).",
# "Extended reality,VR, AR, and mixed reality hardware/software.",
# "High performance computing,Supercomputers for complex simulations.",
# "Mesh and infrastructure-independent networks,Self-relaying device networks without central infrastructure.",
# "Protective cyber security technologies,Tools to secure systems, data, and AI.",
# "Post-quantum cryptography,Encryption resistant to quantum attacks.",
# "Additive manufacturing,Layer-by-layer 3D printing with metals, plastics, etc.",
# "Advanced composite materials,Combined materials for superior strength (e.g., carbon fibre).",
# "Advanced explosives and energetic materials,High-energy substances for rapid release.",
# "Advanced magnets and superconductors,Powerful magnets and zero-resistance materials.",
# "Advanced protection,Protective gear like body armour and hazard-resistant fabrics.",
# "Coatings,Surface treatments for protection or function (e.g., stealth, anti-corrosion).",
# "Continuous flow chemical synthesis,Non-batch production of chemicals/pharma.",
# "Critical minerals extraction and processing,Mining and refining key minerals.",
# "High-specification machining processes,Precision cutting/shaping (e.g., CNC, laser).",
# "Nanoscale materials and manufacturing,Sub-100nm structures with unique properties.",
# "Novel metamaterials,Engineered materials with unnatural properties (e.g., light bending).",
# "Smart materials,Property-changing materials (e.g., shape-memory alloys).",
# "Wide and ultrawide bandgap semiconductors,High-power, high-temperature semiconductors.",
# "Advanced data analytics,Large-scale data processing for insights.",
# "Adversarial AI,Attacks and defences for AI systems.",
# "AI algorithms and hardware accelerators,Models and specialised chips (e.g., GPUs).",
# "Computer vision,AI for interpreting images/video.",
# "Generative AI,AI creating text, images, etc.",
# "Machine learning,Algorithms that learn from data.",
# "Natural language processing,AI for understanding/generating language.",
# "Advanced integrated circuit design and fabrication,Sophisticated chips (e.g., SoCs).",
# "Biological manufacturing,Cell-based production of materials/chemicals.",
# "Brain-computer interfaces,Direct neural links to devices.",
# "Genetic engineering,Gene modification (e.g., CRISPR).",
# "Genomic sequencing and analysis,DNA reading and interpretation.",
# "Neuroprosthetics,Devices restoring neural functions.",
# "Novel antibiotics and antivirals,New infection treatments.",
# "Nuclear medicine and radiation therapy,Radioactive diagnostics/treatments.",
# "Synthetic biology,Engineered biological systems.",
# "Vaccines and medical countermeasures,Rapid response biologics.",
# "Advanced aircraft engines,High-speed engines (e.g., hypersonics).",
# "Drones, swarming and collaborative robots,Uncrewed vehicles and swarms.",
# "Hypersonic detection and tracking,Sensors for fast-moving objects.",
# "Small satellites,Compact satellites under 500kg.",
# "Space launch systems,Rocket technologies for orbit.",
# "Autonomous systems operation technologies,Self-governing machines.",
# "Advanced robotics,Human-like or adaptive robots.",
# "Biofuels,Fuels from biological sources.",
# "Directed energy technologies,Laser/microwave energy transfer or weapons.",
# "Electric batteries,Rechargeable electrochemical storage.",
# "Geoengineering,Large-scale climate interventions (e.g., CO₂ removal).",
# "Grid integration technologies,Smart grid management for renewables.",
# "Hydrogen and ammonia for power,Clean energy carriers.",
# "Nuclear energy,Fission-based power generation.",
# "Nuclear waste management and recycling,Safe handling/reuse of waste.",
# "Photovoltaics,Solar electricity generation.",
# "Supercapacitors,Fast-charge energy storage.",
# "Precision agriculture,Data-driven farming with sensors/AI.",
# "Quantum computing,Computation using quantum properties.",
# "Quantum communication,Secure quantum key distribution.",
# "Quantum sensors,Ultra-precise quantum measurements.",
# "Atomic clocks,Ultra-accurate timekeeping.",
# "Gravitational-force sensors,Gravity variation detection.",
# "Inertial navigation systems,GPS-independent positioning.",
# "Magnetic field sensors,Magnetic detection for navigation/threats.",
# "Multispectral and hyperspectral imaging sensors,Multi-band electromagnetic imaging.",
# "Photonic sensors,Light-based environmental sensing.",
# "Radar,Object detection via radio waves.",
# "Satellite positioning and navigation,GNSS systems.",
# "Sonar and acoustic sensors,Underwater sound-based detection.",
# "Air-independent propulsion,Stealth submarine power without oxygen.",
# "Autonomous underwater vehicles,Self-operating subsea platforms.",
# ]

# # Prepend prompt to texts and embed
# text_inputs = [classification_prompt + text for text in texts]
# text_embeddings = model.encode(text_inputs, normalize_embeddings=True)

# # Embed labels (with the same prompt for consistency)
# label_inputs = [classification_prompt + label for label in labels]
# label_embeddings = model.encode(label_inputs, normalize_embeddings=True)

# # Compute cosine similarities
# similarities = cos_sim(text_embeddings, label_embeddings)

# # Predicted labels
# predicted_indices = torch.argmax(similarities, dim=1)
# predicted_labels = [labels[idx] for idx in predicted_indices]

# # Output results
# for text, preds, scores in zip(texts, predicted_labels, similarities):
#     print(f"Text: {text}")
#     print(f"Max value: {torch.max(scores)}")
#     print(f"Predicted: {preds}")
#     print(f"Scores: {scores.tolist()}\n")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
data = pl.read_parquet(DATA_DIR + "processed/class/*")

In [30]:
print_longer(data.with_columns(pl.col("sim").arr.max()).filter(pl.col("sim") > 0.5), 200)

shape: (767, 4)
┌─────────────────────────────────────────────────────────────────┬──────────┬────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────┐
│ text                                                            ┆ sim      ┆ pred_lab                                               ┆ url                                                            │
│ ---                                                             ┆ ---      ┆ ---                                                    ┆ ---                                                            │
│ str                                                             ┆ f32      ┆ str                                                    ┆ str                                                            │
╞═════════════════════════════════════════════════════════════════╪══════════╪════════════════════════════════════════════════════════╪═════════════════════════════════════════════